In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import idx2numpy
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import PIL
import os
import random
import optuna
import joblib

In [2]:
trainimages = []
trainlabels = []
index = 0
mode = "Train"
Map= {}
emotions = os.listdir(f'emotionfolder/{mode}')
for emotion in emotions:
    Map[index]=emotion

    if emotion=="affectnet" or emotion=="emotionfolder":
        continue
    
    files =os.listdir(f'emotionfolder/{mode}/{emotion}')
    for file in files:
        try:
            filepath =f'emotionfolder/{mode}/{emotion}/{file}'
            trainimages.append(filepath)
            trainlabels.append(index)
        except Exception as e:
            pass
    index+=1
    

In [3]:
testimages = []
testlabels = []
index = 0
mode = 'Test'
emotions = os.listdir(f'emotionfolder/{mode}')
for emotion in emotions:
    if emotion=="affectnet" or emotion=="emotionfolder":
        continue
    
    files =os.listdir(f'emotionfolder/{mode}/{emotion}')
    for file in files:
        try:
            img = f'emotionfolder/{mode}/{emotion}/{file}'
            testimages.append(img)
            testlabels.append(index)
        except Exception as e:
            pass
    index+=1
    

In [4]:
#split is rougly 60/40 right now, need to make it ~80/10/10
trainimages.extend(testimages)
trainlabels.extend(testlabels)

trainimages_final = []
trainlabel_final = []
testimages_final = []
testlabel_final = []
valimages_final = []
vallabels_final = []

In [5]:

for i in range(0, len(trainimages)-1):
    num = random.randint(1,10)

    if 1<=num<=8:
        trainimages_final.append(trainimages[i])
        trainlabel_final.append(trainlabels[i])

    elif num==9:
        testimages_final.append(trainimages[i])
        testlabel_final.append(trainlabels[i])

    else:
        valimages_final.append(trainimages[i])
        vallabels_final.append(trainlabels[i])
        
    

In [6]:
target_size = (224,224)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [7]:
transform_train = transforms.Compose([
    transforms.Resize(target_size), 
    transforms.RandomHorizontalFlip(p=0.5), # 50% chance of a horizontal flip
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
transforms_test = transforms.Compose([
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

In [8]:
class MyDataset(Dataset):
    def __init__(self, imageslink, label,transformation_maker):
        super().__init__()
        self.imageslink = imageslink
        self.label = label
        self.transformation_maker = transformation_maker

    def __len__(self):
        return len(self.label)

    def __getitem__(self,idx):
        img = Image.open(self.imageslink[idx])
        img =  self.transformation_maker(img)

        return img, self.label[idx]

In [9]:
trainDataset = MyDataset(trainimages_final, trainlabel_final, transform_train)
testDataset = MyDataset(testimages_final, testlabel_final, transforms_test)
valDataset = MyDataset(valimages_final, vallabels_final, transforms_test)
trainloader = DataLoader(trainDataset, batch_size=256, shuffle=True, num_workers=10, pin_memory=True)
testloader = DataLoader(testDataset, batch_size=256, shuffle=True, num_workers=10, pin_memory=True)
valloader = DataLoader(valDataset, batch_size=256, shuffle=True, num_workers=10, pin_memory=True)
#8outputs

In [10]:
class CnnArchitecture(nn.Module):
    def __init__(self, is_grayscale, output_size, num_layers, hidden_size,  kernel_size, image_size=96):
        super().__init__()

        self.input_size=1 if is_grayscale else 3
        self.output_size = output_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.imagesize = image_size
        self.kernelsize = kernel_size
        self.map = {3:1, 5:2, 7:3}
        self.Network = []

        for i in range(num_layers):
            if i==0:
                self.Network.append(nn.Conv2d(self.input_size, self.hidden_size, kernel_size=self.kernelsize, padding=self.map[self.kernelsize]))
                self.Network.append(nn.ReLU())
                self.Network.append(nn.MaxPool2d(kernel_size=2, stride=2))

            else:
                self.Network.append(nn.Conv2d(self.hidden_size, self.hidden_size, kernel_size=self.kernelsize, padding=self.map[self.kernelsize]))
                self.Network.append(nn.ReLU())
                self.Network.append(nn.MaxPool2d(kernel_size=2, stride=2))

        self.features = nn.Sequential(*self.Network)
        sampletorch = torch.randn(1, self.input_size, self.imagesize, self.imagesize)
        self.dummy_size = self.features(sampletorch).numel()

        self.classifier = nn.Linear(self.dummy_size, self.output_size)

    def forward(self,x):
        x=self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)#no relu here since criterion and loss will take care of it!
                                                                              

                                                                              
                                                            
                                                
        

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
def objective(trial):
    batch_size = trial.suggest_categorical("batch_size", [32,64,128,256])
    num_layers = trial.suggest_categorical("num_layers", [2,4,6])
    hidden_size = trial.suggest_categorical("hidden_size", [16, 64, 128])
    lr = trial.suggest_categorical("lr", [1e-3, 1e-4, 1e-5])
    epochs = trial.suggest_categorical("epochs", [50, 75, 100])
    weight_decay = trial.suggest_categorical("weight_decay", [1e-2, 1e-3, 1e-4])   
    kernel_size = trial.suggest_categorical("kernel_size", [3,5,7])

    

    
    trainloader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
    valloader = DataLoader(valDataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CnnArchitecture(False, 8, num_layers, hidden_size, kernel_size)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    testloss = []

    
    for i in range(epochs):
        model.train()
        training_loss = 0
        for x,y in trainloader:
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            training_loss+=loss.item()
            loss.backward()
            optimizer.step()
    
    
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for x,y in valloader:
                x = x.to(device)
                y = y.to(device)
                y_pred = model(x)
                loss = criterion(y_pred, y)
                test_loss+=loss.item()
                total += y.size(0)
                probability, predicted = torch.max(y_pred.data, 1)
                correct += (predicted == y).sum().item()
                
        if i%20==0 or i==epochs-1: 
            avg_train_loss = training_loss / len(trainloader)
            avg_test_loss = test_loss / len(valloader)
            accuracy = (100 * correct) / total
    
            print(f"Epoch num: {i}")
            print(f"  Train Loss: {avg_train_loss}")
            print(f"  Test Loss:  {avg_test_loss}")
            print(f"  Accuracy:   {accuracy}%")
            print("--------------------")
            testloss.append(avg_test_loss)
            trial.report(avg_test_loss, i)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return testloss[-1]#want to minimize testloss!


In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40)

[I 2025-11-13 08:34:00,863] A new study created in memory with name: no-name-2ee787cb-024a-491c-93d8-bbf5e09ba882
Exception in thread Thread-5 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.13/threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "/usr/lib/python3.13/threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dilyxs/programming/python/venv/lib/python3.13/site-packages/torch/utils/data/_utils/pin_memory.py", line 52, in _pin_memory_loop
    do_one_step()
    ~~~~~~~~~~~^^
  File "/home/dilyxs/programming/python/venv/lib/python3.13/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.13/multiprocessing/queues.py", line 120, in get
    return _ForkingPickler.loads(res)
           ~~~~~~~~~~~~~~~~~~~~~^^^^^
  File "/home/dilyxs/prog

KeyboardInterrupt: 

In [22]:
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

NameError: name 'study' is not defined

In [13]:
10 * 400 / 600

6.666666666666667

In [213]:
#now build model with best found params
best_params = study.best_params
batch_size = best_params["batch_size"]
num_layers = best_params["num_layers"]
hidden_size = best_params["hidden_size"]
lr = best_params["lr"]
epochs = best_params["epochs"]
kernel_size = best_params["kernel_size"]
weight_decay = best_params["weight_decay"]

In [21]:
#custom dataset with both train && val
compressedtrain = []
compressedtrainlabels = []
compressedtrain.extend(trainimages_final)
compressedtrain.extend(valimages_final)
compressedtrainlabels.extend(trainlabel_final)
compressedtrainlabels.extend(vallabels_final)
compressedDataset =  MyDataset(compressedtrain, compressedtrainlabels, transform_train)



trainloader = DataLoader(compressedDataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CnnArchitecture(False, 8, num_layers, hidden_size, kernel_size)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
testloss = []


for i in range(150):
    model.train()
    training_loss = 0
    for x,y in trainloader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        training_loss+=loss.item()
        loss.backward()
        optimizer.step()


    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for x,y in valloader:
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            test_loss+=loss.item()
            total += y.size(0)
            probability, predicted = torch.max(y_pred.data, 1)
            correct += (predicted == y).sum().item()
            
    if i%20==0 or i==epochs-1: 
        avg_train_loss = training_loss / len(trainloader)
        avg_test_loss = test_loss / len(valloader)
        accuracy = (100 * correct) / total

        print(f"Epoch num: {i}")
        print(f"  Train Loss: {avg_train_loss}")
        print(f"  Test Loss:  {avg_test_loss}")
        print(f"  Accuracy:   {accuracy}%")
        print("--------------------")
        testloss.append(avg_test_loss)

NameError: name 'batch_size' is not defined

In [215]:
joblib.dump(model, "cnn_emotion_prediction.pkl")

['cnn_emotion_prediction.pkl']

In [216]:
device = torch.device('cuda')
model = joblib.load('cnn_emotion_prediction.pkl')
model = model.to(device)

In [ ]:
#now let us test it on testDataset, model is not performant at all, training from scratch is not worth it!

In [217]:
test_loss = 0
total = 0
correct = 0
for x,y in testloader:
    x = x.to(device)
    y = y.to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y)
    test_loss+=loss.item()
    total+=y.size(0)
    _, prediction = torch.max(y_pred, 1)
    correct+=(prediction==y).sum().item()

avg_test_loss = test_loss / len(testloader)
accuracy = (100 * correct) / total

print(f"  Test Loss:  {avg_test_loss}")
print(f"  Accuracy:   {accuracy}%")
    

  Test Loss:  1.7339561184247334
  Accuracy:   27.995971802618328%


In [23]:
#FROM HERE ON OUT, IT IS NOT BUILDING A MODEL FROM SCRATH FOR THIS PROBLEM!

In [13]:
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad=False

/home/dilyxs/programming/python/venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dilyxs/programming/python/venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
lastpreviouslayer = model.fc.in_features
output=8
model.fc = nn.Linear(lastpreviouslayer, output)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [17]:
def objective(trial):
    batch_size=trial.suggest_categorical("batch_size", [128])
    epochs = trial.suggest_categorical("epochs", [16,32,64])
    lr = trial.suggest_categorical("lr", [1e-3, 1e-4, 1e-5])
    weight_decay = trial.suggest_categorical("weight_decay", [1e-5, 1e-7, 0])
    model_size = trial.suggest_categorical("model_size", [18,34,50])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    output=8
    if model_size==18:
        model = torchvision.models.resnet18(pretrained=True)
    elif model_size==32:
        model = torchvision.models.resnet34(pretrained=True)
    else:
        model = torchvision.models.resnet50(pretrained=True)
    lastpreviouslayer = model.fc.in_features

    for param in model.parameters():
        param.requires_grad=False
    model.fc = nn.Linear(lastpreviouslayer, output)
    model = model.to(device)
    

    

    
    trainloader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
    valloader = DataLoader(valDataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    testloss = []

    
    for i in range(epochs):
        model.train()
        training_loss = 0
        for x,y in trainloader:
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            training_loss+=loss.item()
            loss.backward()
            optimizer.step()
    
    
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for x,y in valloader:
                x = x.to(device)
                y = y.to(device)
                y_pred = model(x)
                loss = criterion(y_pred, y)
                test_loss+=loss.item()
                total += y.size(0)
                probability, predicted = torch.max(y_pred.data, 1)
                correct += (predicted == y).sum().item()
                
        if i%20==0 or i==epochs-1: 
            avg_train_loss = training_loss / len(trainloader)
            avg_test_loss = test_loss / len(valloader)
            accuracy = (100 * correct) / total
    
            print(f"Epoch num: {i}")
            print(f"  Train Loss: {avg_train_loss}")
            print(f"  Test Loss:  {avg_test_loss}")
            print(f"  Accuracy:   {accuracy}%")
            print("--------------------")
            testloss.append(avg_test_loss)
            trial.report(avg_test_loss, i)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return testloss[-1]#want to minimize testloss!


In [18]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=3)

[I 2025-11-13 20:10:22,348] A new study created in memory with name: no-name-7b04b167-fa59-48a5-975d-86613d7dcc25


Epoch num: 0
  Train Loss: 2.1084921496609845
  Test Loss:  2.05455673734347
  Accuracy:   20.67951649787651%
--------------------


[I 2025-11-13 20:16:16,548] Trial 0 finished with value: 1.7375248124202092 and parameters: {'batch_size': 128, 'epochs': 16, 'lr': 1e-05, 'weight_decay': 0, 'model_size': 18}. Best is trial 0 with value: 1.7375248124202092.


Epoch num: 15
  Train Loss: 1.7392577069501083
  Test Loss:  1.7375248124202092
  Accuracy:   34.20450833061091%
--------------------


/home/dilyxs/programming/python/venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch num: 0
  Train Loss: 2.041928700481852
  Test Loss:  2.0213674853245416
  Accuracy:   21.006207121855603%
--------------------
Epoch num: 20
  Train Loss: 1.5512117116401594
  Test Loss:  1.5670379102230072
  Accuracy:   43.515191114015025%
--------------------


[I 2025-11-13 20:46:19,806] Trial 1 finished with value: 1.4954314529895782 and parameters: {'batch_size': 128, 'epochs': 32, 'lr': 1e-05, 'weight_decay': 1e-07, 'model_size': 34}. Best is trial 1 with value: 1.4954314529895782.


Epoch num: 31
  Train Loss: 1.4752018054326375
  Test Loss:  1.4954314529895782
  Accuracy:   45.279320483502126%
--------------------
Epoch num: 0
  Train Loss: 1.5701719957093399
  Test Loss:  1.4454486320416133
  Accuracy:   43.64586736360666%
--------------------
Epoch num: 20
  Train Loss: 1.1919004501154025
  Test Loss:  1.3366331507762272
  Accuracy:   48.676902972884676%
--------------------


[I 2025-11-13 21:16:30,621] Trial 2 finished with value: 1.2907888889312744 and parameters: {'batch_size': 128, 'epochs': 32, 'lr': 0.001, 'weight_decay': 1e-05, 'model_size': 50}. Best is trial 2 with value: 1.2907888889312744.


Epoch num: 31
  Train Loss: 1.1486962366228302
  Test Loss:  1.2907888889312744
  Accuracy:   50.31035609278014%
--------------------


In [19]:
study.best_params

{'batch_size': 128,
 'epochs': 32,
 'lr': 0.001,
 'weight_decay': 1e-05,
 'model_size': 50}

In [ ]:
#build the new model!

In [21]:
output=8
batch_size=128
epochs=64
lr=0.001
weight_decay=1e-5
model_size=50
if model_size==18:
    model = torchvision.models.resnet18(pretrained=True)
elif model_size==32:
    model = torchvision.models.resnet34(pretrained=True)
else:
    model = torchvision.models.resnet50(pretrained=True)
lastpreviouslayer = model.fc.in_features

for param in model.parameters():
    param.requires_grad=False
model.fc = nn.Linear(lastpreviouslayer, output)
model = model.to(device)





trainloader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
valloader = DataLoader(valDataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
testloss = []


for i in range(epochs):
    model.train()
    training_loss = 0
    for x,y in trainloader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        training_loss+=loss.item()
        loss.backward()
        optimizer.step()


    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for x,y in valloader:
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            test_loss+=loss.item()
            total += y.size(0)
            probability, predicted = torch.max(y_pred.data, 1)
            correct += (predicted == y).sum().item()
            
    if i%20==0 or i==epochs-1: 
        avg_train_loss = training_loss / len(trainloader)
        avg_test_loss = test_loss / len(valloader)
        accuracy = (100 * correct) / total

        print(f"Epoch num: {i}")
        print(f"  Train Loss: {avg_train_loss}")
        print(f"  Test Loss:  {avg_test_loss}")
        print(f"  Accuracy:   {accuracy}%")
        print("--------------------")
        testloss.append(avg_test_loss)


Epoch num: 0
  Train Loss: 1.562836433450381
  Test Loss:  1.4063928922017415
  Accuracy:   46.128716105847765%
--------------------
Epoch num: 20
  Train Loss: 1.1974625547106068
  Test Loss:  1.298115645845731
  Accuracy:   50.44103234237178%
--------------------
Epoch num: 40
  Train Loss: 1.1375562998776634
  Test Loss:  1.2693762977917988
  Accuracy:   51.38843515191114%
--------------------
Epoch num: 60
  Train Loss: 1.1072659293810527
  Test Loss:  1.2743211040894191
  Accuracy:   50.40836327997386%
--------------------
Epoch num: 63
  Train Loss: 1.1021915295471747
  Test Loss:  1.2686743239561717
  Accuracy:   51.09441359032996%
--------------------


In [23]:
joblib.dump(model, "resnet_fine_tuned.pkl")

['resnet_fine_tuned.pkl']

In [174]:
#testrun for prediction
img = Image.open('emotionfolder/Train/anger/image0022969.jpg')
img = transforms_test(img)
img = img.unsqueeze(0)
img = img.to(device)
model.eval()
with torch.no_grad():
    y_pred = model(img)

In [188]:

EmotionMap = {v:k for k,v in Map.items()}

In [189]:
EmotionMap

{'anger': 0,
 'contempt': 1,
 'disgust': 2,
 'fear': 3,
 'happy': 4,
 'neutral': 5,
 'sad': 6,
 'surprise': 7}

In [192]:
#need to define this into helper file later on...
def Predict(imgpath, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    PredictionEmotionMap =EmotionMap.copy()
    img = Image.open(imgpath)
    img = transforms_test(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        y_pred = model(img)

    prediction_confidence_array = torch.nn.functional.softmax(y_pred, dim=1)[0].cpu().numpy()
    for i in range(len(prediction_confidence_array)):
        emotion = Map[i]
        PredictionEmotionMap[emotion] = prediction_confidence_array[i]

    return PredictionEmotionMap
    #how would I convert confidence to a 100%scale cause it ain't like that right now no?
    

In [193]:
Predict('emotionfolder/Train/anger/image0022969.jpg', model)

{'anger': np.float32(0.10275879),
 'contempt': np.float32(0.10070733),
 'disgust': np.float32(0.09365553),
 'fear': np.float32(0.10301596),
 'happy': np.float32(0.16819917),
 'neutral': np.float32(0.15946205),
 'sad': np.float32(0.13645828),
 'surprise': np.float32(0.13574292)}